# Analyse de la qualité des données du fichier Ventes_Q1_2025.csv


## Résumé des anomalies observées

## Champs ville
- 17% de valeurs manquantes (83% de complétude)

## Champ date
- 25% des lignes correspondent à la période Q1 (1er trimestre)

## Champ prix
- 9% de prix négatifs (91% de prix valides)

## Champ quantité
- 2,2% de quantités négatives (98% de quantités valides)

## Analyse croisée avec les fichiers Produit et MarketPlace
- les prix négatifs ne sont pas dû à un transfert d'anomalie depuis ce fichier
- les chiffres d'affaires négatifs ne correspondent pas à des produits retournés sur le canal Web (0,7% de corrélation)


### Points positifs
- ✅ **Aucun doublon** détecté dans le dataset

In [2]:
import pandas as pd
import numpy as np

In [3]:
df_ventes = pd.read_csv('../data/raw/Ventes_Q1_2025.csv')

In [4]:
print("AUDIT DATAFRAME VENTES 1er TRIMESTRE 2025")

display(df_ventes.head())
display(df_ventes.info())

print("\n---Taux de valeurs manquantes (%)---")
missing_percentage = df_ventes.isnull().sum() / len(df_ventes) * 100
print(missing_percentage[missing_percentage > 0].sort_values(ascending=False))

print("\n--- Vérification des doublons ---")
clients_doublons = df_ventes.duplicated().sum()
print(f"Nombre total de doublons: {clients_doublons} / soit {clients_doublons/len(df_ventes) * 100:.2f}%")


print("\n--- Vérification des doublons sur le order_id ---")
nb_doublons_order_id = df_ventes['order_id'].duplicated().sum()
print(f"Nombre de order_id en doublon : {nb_doublons_order_id}")

AUDIT DATAFRAME VENTES 1er TRIMESTRE 2025


,order_id,date,client_id,produit,categorie,prix_unitaire,quantite,ville,canal,chiffre_affaires
0,1,2025-03-05,391,Batterie externe,Informatique,56.80,2,Saint ConstanceVille,Web,113.60
1,2,2025-01-26,261,Webcam HD,Informatique,146.42,-1,NaN,Web,-146.42
2,3,2025-08-03,461,Disque SSD,Informatique,11.44,3,Leleu-sur-Lopez,Boutique,34.32
3,4,2025-07-12,152,Batterie externe,Audio,138.78,2,Saint Lucie,Boutique,277.56
4,5,2025-04-18,107,Disque SSD,Informatqiue,9.88,5,Toussaint,Web,49.40


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   order_id          2000 non-null   int64  
 1   date              2000 non-null   object 
 2   client_id         2000 non-null   int64  
 3   produit           2000 non-null   object 
 4   categorie         2000 non-null   object 
 5   prix_unitaire     2000 non-null   float64
 6   quantite          2000 non-null   int64  
 7   ville             1651 non-null   object 
 8   canal             2000 non-null   object 
 9   chiffre_affaires  2000 non-null   float64
dtypes: float64(2), int64(3), object(5)
memory usage: 156.4+ KB


None


---Taux de valeurs manquantes (%)---
ville    17.45
dtype: float64

--- Vérification des doublons ---
Nombre total de doublons: 0 / soit 0.00%

--- Vérification des doublons sur le order_id ---
Nombre de order_id en doublon : 0


In [5]:
print("\n--- Vérification de la cohérence temporelle de l'export ---")

# Convertir les dates si ce n'est pas déjà fait
df_ventes['date'] = pd.to_datetime(df_ventes['date'], errors='coerce')

# Identifier la période attendue d'après le nom du fichier
periode_attendue_debut = pd.Timestamp('2025-01-01')
periode_attendue_fin = pd.Timestamp('2025-03-31')
print(f"Période attendue d'après le fichier : Mars 2025")
print(f"  Du {periode_attendue_debut.strftime('%d/%m/%Y')} au {periode_attendue_fin.strftime('%d/%m/%Y')}")

# Analyser les dates réelles présentes dans le fichier
date_min = df_ventes['date'].min()
date_max = df_ventes['date'].max()

print(f"\nPériode réelle dans les données :")
print(f"  Date la plus ancienne : {date_min.strftime('%d/%m/%Y') if pd.notna(date_min) else 'NaT'}")
print(f"  Date la plus récente : {date_max.strftime('%d/%m/%Y') if pd.notna(date_max) else 'NaT'}")

# Identifier les lignes hors période
mask_avant = df_ventes['date'] < periode_attendue_debut
mask_apres = df_ventes['date'] > periode_attendue_fin
mask_hors_periode = mask_avant | mask_apres

nb_hors_periode = mask_hors_periode.sum()
nb_total = len(df_ventes)
pourcentage_hors_periode = (nb_hors_periode / nb_total) * 100

if nb_hors_periode > 0:
    print(f"\n⚠️ ANOMALIE DÉTECTÉE : {nb_hors_periode} lignes ({pourcentage_hors_periode:.2f}%) ont des dates hors période !")

    # Détailler les anomalies
    nb_avant = mask_avant.sum()
    nb_apres = mask_apres.sum()

    print(f"  - Dates avant janvier 2025 : {nb_avant}")
    print(f"  - Dates après mars 2025 : {nb_apres}")

    # Créer un DataFrame des anomalies
    df_hors_periode = df_ventes[mask_hors_periode].copy()
    df_hors_periode = df_hors_periode.sort_values('date')

    print("\n--- Exemples de lignes avec dates hors période ---")
    display(df_hors_periode[['order_id','date', 'produit', 'quantite', 'prix_unitaire','canal']].head(10))

else:
    print(f"\n✓ Toutes les dates correspondent à la période attendue (Mars 2025)")

    # Vérifier les dates NaT (non converties)
nb_nat = df_ventes['date'].isna().sum()
if nb_nat > 0:
    print(f"\n⚠️ {nb_nat} dates invalides (NaT) détectées")



--- Vérification de la cohérence temporelle de l'export ---
Période attendue d'après le fichier : Mars 2025
  Du 01/01/2025 au 31/03/2025

Période réelle dans les données :
  Date la plus ancienne : 29/10/2024
  Date la plus récente : 29/10/2025

⚠️ ANOMALIE DÉTECTÉE : 1494 lignes (74.70%) ont des dates hors période !
  - Dates avant janvier 2025 : 346
  - Dates après mars 2025 : 1148

--- Exemples de lignes avec dates hors période ---


,order_id,date,produit,quantite,prix_unitaire,canal
723,724,2024-10-29,Webcam HD,2,59.91,Boutique
354,355,2024-10-29,Chargeur rapide,2,120.56,Web
899,900,2024-10-29,Webcam HD,1,6.67,Boutique
1167,1168,2024-10-29,Enceinte Bluetooth,1,-103.78,Web
1788,1789,2024-10-29,Chargeur rapide,2,109.85,Web
764,765,2024-10-30,Clavier,0,136.57,Web
263,264,2024-10-30,Câble USB-C,1,61.01,Marketplace
1550,1551,2024-10-30,Disque SSD,2,-180.48,Web
520,521,2024-10-30,Clavier,1,88.22,Web
562,563,2024-10-30,Casque Bluetooth,1,153.48,Web


In [6]:
print("\n---Vérification des prix négatifs---")
nb_prix_negatifs = (df_ventes['prix_unitaire']<0).sum()
print("Nombre total de prix négatifs :",nb_prix_negatifs)
print(f"Pourcentage de prix négatifs : {nb_prix_negatifs/len(df_ventes) * 100:.2f}%")

lignes_prix_negatifs = df_ventes[df_ventes['prix_unitaire'] < 0]
print("\nExemples de lignes avec des prix négatifs :")
display(lignes_prix_negatifs.head(8))

# Identifier les lignes avec prix négatifs
mask_prix_negatifs = df_ventes['prix_unitaire'] < 0

# Compter les prix négatifs par canal
prix_negatifs_par_canal = df_ventes[mask_prix_negatifs]['canal'].value_counts()
print("\n--- Détail avec pourcentage ---")
for canal, count in prix_negatifs_par_canal.items():
    pourcentage = (count / mask_prix_negatifs.sum() * 100)
    print(f"{canal}: {count} prix négatifs ({pourcentage:.1f}%)")


---Vérification des prix négatifs---
Nombre total de prix négatifs : 183
Pourcentage de prix négatifs : 9.15%

Exemples de lignes avec des prix négatifs :


,order_id,date,client_id,produit,categorie,prix_unitaire,quantite,ville,canal,chiffre_affaires
7,8,2025-03-04,461,Webcam HD,Informatique,-15.16,1,Blin-la-Forêt,Web,-15.16
18,19,2024-12-27,496,Webcam HD,Informatique,-16.44,3,Dupuy-la-Forêt,Web,-49.32
44,45,2025-09-13,182,Écran 24 pouces,Informatique,-16.71,1,Clémentnec,Web,-16.71
48,49,2024-12-31,297,Enceinte Bluetooth,Informatique,-93.23,3,Lamy,Web,-279.69
53,54,2025-01-29,31,Enceinte Bluetooth,Informatqiue,-24.80,2,Salmon,Boutique,-49.60
59,60,2025-06-19,357,Clavier,Informatique,-67.01,1,Valette,Web,-67.01
75,76,2024-12-22,334,Disque SSD,Informatique,-161.49,1,Millet,Web,-161.49
94,95,2025-10-07,354,Webcam HD,Audio,-17.12,1,Regnier-sur-Nicolas,Web,-17.12



--- Détail avec pourcentage ---
Web: 113 prix négatifs (61.7%)
Boutique: 51 prix négatifs (27.9%)
Marketplace: 19 prix négatifs (10.4%)


In [7]:
print("\n---Vérification des quantités négatives---")
nb_prix_negatifs = (df_ventes['quantite']<0).sum()
print("Nombre total de quantités négatives :",nb_prix_negatifs)
print(f"Pourcentage de quantités négatives : {nb_prix_negatifs/len(df_ventes) * 100:.2f}%")

lignes_quantites_negatives = df_ventes[df_ventes['quantite'] < 0]
print("\nExemples de lignes avec des quantités négatives :")
display(lignes_quantites_negatives.head(8))

# Identifier les lignes avec prix négatifs
mask_quantite_negatives = df_ventes['quantite'] < 0

# Compter les prix négatifs par canal
prix_negatifs_par_canal = df_ventes[mask_quantite_negatives]['canal'].value_counts()
print("\n--- Détail avec pourcentage ---")
for canal, count in prix_negatifs_par_canal.items():
    pourcentage = (count / mask_quantite_negatives.sum() * 100)
    print(f"{canal}: {count} quantite negatives ({pourcentage:.1f}%)")


---Vérification des quantités négatives---
Nombre total de quantités négatives : 44
Pourcentage de quantités négatives : 2.20%

Exemples de lignes avec des quantités négatives :


,order_id,date,client_id,produit,categorie,prix_unitaire,quantite,ville,canal,chiffre_affaires
1,2,2025-01-26,261,Webcam HD,Informatique,146.42,-1,NaN,Web,-146.42
24,25,2025-09-08,213,Écran 24 pouces,Informatique,99.16,-1,NaN,Boutique,-99.16
104,105,2024-12-08,26,Câble USB-C,Audio,102.70,-1,Aubry-sur-Rivière,Web,-102.70
209,210,2025-09-22,347,Casque Bluetooth,Informatique,163.15,-1,Regnier-sur-Nicolas,Web,-163.15
266,267,2024-11-06,455,Souris,Informatique,90.89,-1,Colinnec,Web,-90.89
357,358,2025-04-11,449,Souris,Accessoire,90.29,-1,Sainte Isabelle,Boutique,-90.29
376,377,2025-07-14,200,Webcam HD,Audio,28.29,-1,Diallodan,Web,-28.29
404,405,2025-06-04,58,Batterie externe,Informatqiue,123.38,-1,Saint HenrietteVille,Web,-123.38



--- Détail avec pourcentage ---
Web: 29 quantite negatives (65.9%)
Boutique: 12 quantite negatives (27.3%)
Marketplace: 3 quantite negatives (6.8%)


In [8]:
print("\n--- Analyse des catégories de produits ---")

# Nombre de catégories uniques
nb_categories = df_ventes['categorie'].nunique()
print(f"Nombre de catégories uniques : {nb_categories}")
print("\n--- Répartition des catégories ---")
categories_count = df_ventes['categorie'].value_counts(dropna=False)

# Pourcentage par catégorie
categories_pct = df_ventes['categorie'].value_counts(dropna=False, normalize=True) * 100
for cat, pct in categories_pct.items():
    count = categories_count[cat]
    print(f"  {cat} : {count} ventes ({pct:.2f}%)")


--- Analyse des catégories de produits ---
Nombre de catégories uniques : 4

--- Répartition des catégories ---
  Informatique : 1159 ventes (57.95%)
  Audio : 410 ventes (20.50%)
  Informatqiue : 216 ventes (10.80%)
  Accessoire : 215 ventes (10.75%)


In [9]:
# Export des données anormales : prix et quantité négatifs pour analyse croisée avec l'export Marketplace
print("\n--- SAUVEGARDE DU FICHIER AVEC LES PRIX ET QUANTITES NEGATIFS ---")
mask_CA_negatif = df_ventes['chiffre_affaires'] < 0
df_ventes_ca_negatifs = df_ventes[mask_CA_negatif].copy()
df_ventes_ca_negatifs.to_csv('../data/processed/Ventes_Q1_2025_CA_negatifs.csv', index=False)


--- SAUVEGARDE DU FICHIER AVEC LES PRIX ET QUANTITES NEGATIFS ---


In [12]:
# import du df marketplace pour analyse croisée
df_marketplace_corrected = pd.read_csv('../data/processed/Marketplace_Export_Q1_2025.csv')

In [16]:
print("\n--- FILTRAGE CANAL WEB ---")

# Filtrer uniquement le canal "Web"
df_ventes_ca_negatifs_web = df_ventes_ca_negatifs[df_ventes_ca_negatifs['canal'] == 'Web'].copy()
print(f"Nombre de lignes avec canal 'Web' : {len(df_ventes_ca_negatifs_web)}")

print("\n--- ANALYSE DES CORRESPONDANCES AVEC MARKETPLACE ---")

# Fusionner avec correspondance order_id = order_id_market
df_fusion = df_ventes_ca_negatifs_web.merge(
    df_marketplace_corrected[['order_id_market', 'statut_retour']],
    left_on='order_id',
    right_on='order_id_market',
    how='left'
)

total = len(df_ventes_ca_negatifs_web)

# Créer le tableau des statistiques
stats = df_fusion['statut_retour'].value_counts(dropna=False)
pourcentages = (stats / total * 100).round(1)

df_resultats = pd.DataFrame({
    'Statut': stats.index.fillna('Aucune correspondance'),
    'Nombre': stats.values,
    'Pourcentage': pourcentages.values
})

print(f"Total analysé : {total} lignes\n")
display(df_resultats)


--- FILTRAGE CANAL WEB ---
Nombre de lignes avec canal 'Web' : 139

--- ANALYSE DES CORRESPONDANCES AVEC MARKETPLACE ---
Total analysé : 139 lignes



,Statut,Nombre,Pourcentage
0,Aucune correspondance,133,95.7
1,Livré,3,2.2
2,En attente,2,1.4
3,Retourné,1,0.7


In [9]:
# copie du dataframe original
df_ventes_clean = df_ventes.copy()

In [10]:
# print("\n--- SAUVEGARDE DU FICHIER VENTES AVEC CORRECTIONS ---")

# # Sauvegarder dans le dossier processed
# df_ventes_clean.to_csv('../data/processed/Ventes_Q1_2025_corrected.csv', index=False)